# Refuerzo de clases minoritarias y ampliación con ZTF

In [ ]:
# Configuración general para evitar errores de warnings y compatibilidad
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["RICH_NO_RICH"] = "1"
print("Configuración de entorno aplicada.")

In [ ]:
# ...existing code for ZTF and minor classes reinforcement...

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from src.utils.inspect_and_export_summary import inspect_and_export_summary

CURVES_DIR = Path("F:/TFM/ztf_curves")
OUTPUT_PARQUET = Path("data/processed/dataset_ztf_labeled.parquet")
CLASES_OBJETIVO = ["Delta Scuti", "Rotational", "Irregular", "Cataclysmic", "White Dwarf", "RR Lyrae", "Young Stellar Object", "Variable"]

csvs = list(CURVES_DIR.glob("*.csv"))
dfs = []
n_valid = 0
n_empty = 0
n_corrupt = 0

print(f"🔎 Procesando {len(csvs)} archivos CSV...")
for path in tqdm(csvs, desc="Consolidando curvas"):
    try:
        if path.stat().st_size == 0:
            n_empty += 1
            path.unlink()
            continue
        df = pd.read_csv(path)
        if "clase_variable_normalizada" in df.columns:
            if df["clase_variable_normalizada"].iloc[0] in CLASES_OBJETIVO:
                dfs.append(df)
                n_valid += 1
    except Exception:
        n_corrupt += 1
        try:
            path.unlink()
        except Exception:
            pass

print(f"✅ Archivos válidos: {n_valid} | 🗑️ Vacíos: {n_empty} | 🛑 Corruptos: {n_corrupt}")

if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_parquet(OUTPUT_PARQUET, index=False)
    print(f"✅ Consolidado: {len(df_final)} filas → {OUTPUT_PARQUET}")
    inspect_and_export_summary(OUTPUT_PARQUET, output_format="csv")
    print("📊 Sumario generado.")
else:
    print("⚠️ No se encontraron curvas válidas para las clases objetivo.")

🔎 Procesando 114136 archivos CSV...


Consolidando curvas: 100%|██████████| 114136/114136 [38:10<00:00, 49.83it/s] 


✅ Archivos válidos: 114134 | 🗑️ Vacíos: 0 | 🛑 Corruptos: 0
✅ Consolidado: 12551038 filas → data\processed\dataset_ztf_labeled.parquet

📁 Inspeccionando: data\processed\dataset_ztf_labeled.parquet


🧮 Procesando por lotes: 96it [05:29,  3.43s/it]


✅ Resumen exportado a: data/processed/summary\dataset_ztf_labeled_summary.csv
📊 Sumario generado.
